In [ ]:
# Import python modules and packages
import csv
import pandas as pd
import re

In [ ]:
# Read in the raw data from the .csv file for the nursing homes and performance
nursing_df = pd.read_csv('../Resources/Provider_Info.csv')

# Verify the data was imported
print(f'Shape of DataFrame: {nursing_df.shape}')
nursing_df

In [ ]:
# Testing for duplicate rows
print(f'Initial number of rows: {nursing_df.shape[0]}')
nursing_df = nursing_df.drop_duplicates()
print(f'Number of rows after dropping duplicate rows: {nursing_df.shape[0]}')

In [ ]:
# Testing for uniqueness of 'Federal Provider Number'
print(f'Number of rows: {nursing_df.shape[0]}')
print(f'Number of unique "Federal Provider Number"s: {len(nursing_df["Federal Provider Number"].unique())}')

In [ ]:
# Because the the resulting numbers are the same in the cell above, we can conclude that there are no 
# duplicate "Federal Provider Number"s and so it can be used as a primary key. 

In [ ]:
# Show all the columns
nursing_df.columns

In [ ]:
# Investigate the types of the columns that are present
nursing_df.dtypes

In [ ]:
# Investigate the count of non-nan values for the columns
print(nursing_df.count().loc[nursing_df.count() < nursing_df.shape[0]])

In [ ]:
# Because location plays an important role in decision making for all sorts of reasons
# (e.g. weather, taxes, large city or rural area, proximity to family, ... etc.), any 
# nursing home without a location associated with it will be dropped from consideration
no_nan_n_df = nursing_df.dropna(subset = ['Location'])
no_nan_n_df.count()

In [ ]:
# Extract coordinates from the 'Location' column and insert 'Null' if there are no
# coordinates listed for a location.
coord_n_df = no_nan_n_df.copy()
coord_n_df['Coordinates'] = [re.search('\([-]?\d+\.\d+, [-]?\d+\.\d+\)', row).group() if len(re.findall('\([-]?\d+\.\d+, [-]?\d+\.\d+\)', row)) > 0 else 'Null' for row in no_nan_n_df['Location'] ]

In [ ]:
# Verify that the column Coordinates was added
print(coord_n_df.shape)
coord_n_df[['Federal Provider Number', 'Location', 'Coordinates']]

In [ ]:

# Remove all rows that have a 'Null' for their 'Coordinates' value primarily b/c
# the time comsumed for tracking down why coordinates couldn't be determined is beyond
# the limits of this project's time horizon.
coord_n_df = coord_n_df.loc[coord_n_df['Coordinates'] != 'Null']

#Verify the any 'Null' Location rows were removed
print(coord_n_df.shape)
coord_n_df.reset_index(inplace=True)
coord_n_df.head(10)

In [ ]:

# Remove extra column created by reset_index
del coord_n_df['level_0']
coord_n_df

In [ ]:
# Extract the Latitude from the Coordinates column and create a Latitude column with the value
# extracted
coord_n_df['Latitude'] = [re.search('[-]?\d+\.\d+', row).group() for row in coord_n_df['Coordinates']]
coord_n_df.head()
coord_n_df[['Federal Provider Number', 'Location', 'Coordinates', 'Latitude']]

In [ ]:
# Extract the Longitude from the Coordinates column and create a Longitude column with the value
# extracted
coord_n_df['Longitude'] = [re.findall('[-]?\d+\.\d+', row)[1] for row in coord_n_df['Coordinates']]
print(coord_n_df.shape)
coord_n_df[['Federal Provider Number', 'Location', 'Coordinates', 'Latitude', 'Longitude']]

In [ ]:
# State the number of columns before deleting a column, so as to verify the
# column deletion after the fact.
len(coord_n_df.columns)

In [ ]:
# Deleting the redundant Coordinates column (Latitude and Longitude columns
# make it redundant).
del coord_n_df['Coordinates']

In [ ]:

# Verify that the column count reduced by 1
len(coord_n_df.columns)

In [ ]:

# Verify that the Coordinates column is gone and the rest of the dataframe is 
# unaffected
coord_n_df.head()

In [ ]:
# Print all columns as an aid in deciding which columns to keep and which to
# truncate from the dataframe
print(len(coord_n_df.columns))
coord_n_df.columns


In [ ]:
# Remove extraneous columns
list_of_columns_to_keep = ['Federal Provider Number', 'Provider Name', 
    'Provider City', 'Provider State', 'Provider Zip Code', 'Provider County Name',
    'Ownership Type', 'Number of Certified Beds', 'Number of Residents in Certified Beds', 
    'Provider Type', 'Provider Resides in Hospital', 
    'Most Recent Health Inspection More Than 2 Years Ago', 
    'Automatic Sprinkler Systems in All Required Areas', 'Overall Rating',
    'Health Inspection Rating','Staffing Rating','RN Staffing Rating',
    'Total Weighted Health Survey Score','Number of Facility Reported Incidents',
    'Number of Substantiated Complaints', 'Number of Fines',
    'Total Amount of Fines in Dollars', 'Number of Payment Denials',
    'Total Number of Penalties', 'Location', 'Processing Date', 'Latitude',
    'Adjusted Total Nurse Staffing Hours per Resident per Day', 'Longitude']

truncated_nursing_df = pd.DataFrame()
truncated_nursing_df = coord_n_df[list_of_columns_to_keep]


In [ ]:
# Assure that the columns to keep were kept and the columns to remove
# were removed
print(len(truncated_nursing_df.columns))
truncated_nursing_df.columns

In [ ]:
# Calculating R Coefficients between columns
correlations_df = truncated_nursing_df.corr()
correlations_df

In [ ]:
correlations_df.insert(0, 'Column Of Category', correlations_df.index)
correlations_df

In [ ]:

# Save correlations to file
correlations_df.to_json('./correlations_df_by_record.json', orient='records')

In [ ]:
# Create a saving point (for the loading point in the cell below)
truncated_nursing_df.to_json('./truncated_nursing_df1.json')
truncated_nursing_df.to_json('./truncated_nursing_df1_by_record.json', orient='records')
truncated_nursing_df.to_csv('./truncated_nursing_df1.csv', index=False)

In [ ]:
# Create a loading point to skip all the previous cells when rerunning
# the code further past this cell
truncated_nursing_df = pd.read_csv('./truncated_nursing_df1.csv')

In [ ]:
# Verify that the load of the data was successful
print(truncated_nursing_df.shape)
truncated_nursing_df.head()

In [ ]:
# Create the a performance table
list_of_performance_columns_df = ['Most Recent Health Inspection More Than 2 Years Ago', 'Overall Rating',
       'Health Inspection Rating', 'Staffing Rating', 'RN Staffing Rating',
       'Total Weighted Health Survey Score',
       'Number of Facility Reported Incidents',
       'Number of Substantiated Complaints', 'Number of Fines',
       'Total Amount of Fines in Dollars', 'Number of Payment Denials',
       'Total Number of Penalties']

performance_table_df = pd.DataFrame()
performance_table_df[list_of_performance_columns_df] = truncated_nursing_df[list_of_performance_columns_df]

# Verify that the performance table was created
print(performance_table_df.shape)
performance_table_df.head()

In [ ]:
# Show the number of rows in performance_table_df just before any duplicate rows are
# dropped to verify that rows were dropped.
performance_table_df.shape

In [ ]:
# Remove dupulicate rows
performance_table_df.drop_duplicates(subset=list_of_performance_columns_df, inplace=True)

In [ ]:
# Verify that rows were dropped.
print(performance_table_df.shape)

In [ ]:
# Create a primary key for performance_table_df called 'performance_id'. 
performance_table_df['performance_id'] = [i for i in range(performance_table_df.shape[0])]

# Verify that 'performance_id' was created
print(performance_table_df.shape)
performance_table_df.head()

In [ ]:
# Save performance table data to a .csv and .json (for the loading point in the cell below and 
# for data manipulations, respectively)
performance_table_df.to_csv('../TableData/performance_table.csv', index=False)
performance_table_df.to_json('../TableData/performance_table.json')

In [ ]:
# Create a loading point to skip all the previous cells when rerunning
# the code further past this cell
performance_table_df = pd.read_csv('../TableData/performance_table.csv')

In [ ]:
# Verify that the load of the data was successful
print(performance_table_df.shape)
performance_table_df.head()

In [ ]:
# Show the shape of truncated_nursing_df just before adding a column for verification
# after the column is added
truncated_nursing_df.shape

In [ ]:
# Add a column for the foreign key called performance_id in what will be used
# to create the business_table_df further below and initialize performance_id
# to an unused and recognizable value (-1)
truncated_nursing_df['performance_id'] = -1

In [ ]:

# Verify that the column was added
print(truncated_nursing_df.shape)
truncated_nursing_df[['Provider Name', 'performance_id']]

In [ ]:
# Add corrected performance_id foriegn key to truncated_nursing_df
for i in range(truncated_nursing_df.shape[0]):
    for j in range(performance_table_df.shape[0]):
        all_performance_columns_match = True
        for k in range(len(list_of_performance_columns_df)):

            # Must convert the value to str otherwise the 'nan' of truncated_nursing_df can't
            # match the 'nan' of performance_table_df
            if str(truncated_nursing_df[list_of_performance_columns_df[k]].iloc[i]) != str(performance_table_df[list_of_performance_columns_df[k]].iloc[j]):
                all_performance_columns_match = False
                break
        if  all_performance_columns_match:     
            truncated_nursing_df['performance_id'].iloc[i] = performance_table_df['performance_id'].iloc[j]
            break



In [ ]:
# Verify that foreign keys 'perfomance_id' are no longer '-1'
print(len(truncated_nursing_df[truncated_nursing_df['performance_id'] != '-1']))
print(truncated_nursing_df.shape[0])
truncated_nursing_df[['Provider Name', 'performance_id']]

In [ ]:
# Show that record 407 of the truncated table and row 16 of the performance table have the
# same values for the columns of performance table (list_of_performance_columns_df) and
# have the same performance_id (16) associated with them.
print(truncated_nursing_df[list_of_performance_columns_df].iloc[407])
print(f"performance_id for truncated is: {truncated_nursing_df['performance_id'].iloc[407]}")
performance_table_df.iloc[16]

In [ ]:
# Show the shape of truncated_nursing_df before the save point to compare
# with its shape after the save point
truncated_nursing_df.shape

In [ ]:
# Create a saving point (for the loading point in the cell below)
truncated_nursing_df.to_csv('./truncated_nursing_df2.csv', index=False)
truncated_nursing_df.to_json('./truncated_nursing_df2_by_record.json', orient='records')
truncated_nursing_df.to_json('./truncated_nursing_df2.json')


In [ ]:
# Create a loading point to skip all the previous cells when rerunning
# the code further past this cell
truncated_nursing_df = pd.read_csv('./truncated_nursing_df2.csv')

In [ ]:
# Verify that the load of the data was successful
print(truncated_nursing_df.shape)
truncated_nursing_df.head()

In [ ]:
# Create the a business table
list_of_business_columns = ['Federal Provider Number', 'Provider Name', 
    'Provider City', 'Provider Zip Code', 'Provider County Name',
    'Ownership Type', 'Number of Certified Beds', 'Number of Residents in Certified Beds', 
    'Provider Type', 'Provider Resides in Hospital',  
    'Automatic Sprinkler Systems in All Required Areas', 'Location', 'Processing Date', 
    'Latitude','Longitude', 'Adjusted Total Nurse Staffing Hours per Resident per Day',
    'performance_id']
business_table_df = pd.DataFrame()
business_table_df[list_of_business_columns] = truncated_nursing_df[list_of_business_columns] 

# Verify that the business table was created
print(business_table_df.shape)
business_table_df.head()

In [ ]:
# Save business table and create a saving point (for the loading point in the cell below)
business_table_df.to_csv('../TableData/business_table.csv', index=False)
business_table_df.to_json('../TableData/business_table.json')

In [ ]:
# Create a loading point to skip all the previous cells when rerunning
# the code further past this cell
business_table_df = pd.read_csv('../TableData/business_table.csv')

In [ ]:
# Verify that the load of the data was successful
print(business_table_df.shape)
business_table_df.head()

In [ ]:
# Create a zipcode table
list_of_state_zipcodes = ['Provider State', 'Provider Zip Code']
zip_state_table_df = pd.DataFrame()
zip_state_table_df[list_of_state_zipcodes] = truncated_nursing_df[list_of_state_zipcodes]

# Verify that the load of the data was successful
print(zip_state_table_df.shape)
zip_state_table_df.head()

In [ ]:
# Show number of rows before dropping the duplicates
print(zip_state_table_df.shape[0])

In [ ]:
# Drop duplicate rows
zip_state_table_df.drop_duplicates(inplace=True)

In [ ]:
# Verify that rows were dropped
print(zip_state_table_df.shape)
zip_state_table_df.head()

In [ ]:
# Save zipcode table and create a saving point (for the loading point in the cell below)
zip_state_table_df.to_csv('../TableData/zipcode_table.csv', index=False)
zip_state_table_df.to_json('../TableData/zipcode_table.json')

In [ ]:
# Create a loading point to skip all the previous cells when rerunning
# the code further past this cell
zip_state_table_df = pd.read_csv('../TableData/zipcode_table.csv')

In [ ]:
# Verify that the load of the data was successful
print(zip_state_table_df.shape)
zip_state_table_df.head()